In [68]:
import os
import sys
import numpy as np
import pandas as pd
import re
import math
match_number = re.compile('-?\ *[0-9]+\.?[0-9]*(?:[Ee]\ *-?\ *[0-9]+)?')

In [37]:
df = pd.read_csv('surface_data-t=5.0.csv')
df

,p,dUdt:0,dUdt:1,dUdt:2,U:0,U:1,U:2,grad(U):0,grad(U):1,grad(U):2,...,grad(U):5,grad(U):6,grad(U):7,grad(U):8,Area,Normals:0,Normals:1,Normals:2,Cell ID,Block Number
0,0.917458,0.000000,0.000002,-0.000065,0.000000,-0.000002,0.004778,-0.408749,0.0,0.0,...,682.583,-0.049298,0.108494,0.5016,4.375000e-11,1,0,0,0,2
1,0.917458,0.000000,0.000002,-0.000065,0.000000,-0.000002,0.004778,-0.408749,0.0,0.0,...,628.559,-0.049298,0.108494,0.5016,4.375000e-11,1,0,0,1,2
2,0.917991,0.000000,0.000006,-0.000169,0.000000,-0.000009,0.013578,-0.965296,0.0,0.0,...,1159.390,-0.095318,0.156220,0.7920,4.375000e-11,1,0,0,2,2
3,0.918500,0.000000,0.000009,-0.000241,0.000000,-0.000013,0.021010,-1.324450,0.0,0.0,...,979.950,-0.116568,0.156112,0.8720,4.375000e-11,1,0,0,3,2
4,0.918913,0.000000,0.000011,-0.000292,0.000000,-0.000015,0.027297,-1.567000,0.0,0.0,...,829.293,-0.126608,0.118696,0.8960,4.375000e-11,1,0,0,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9679,0.000000,-0.000056,0.000076,-0.002668,0.000185,-0.000098,0.136725,-22.161400,0.0,0.0,...,0.000,-0.800080,-0.073112,9.6000,8.777990e-11,0,0,-1,1381,6
9680,0.000000,-0.000056,0.000076,-0.002668,0.000185,-0.000098,0.136725,-22.161400,0.0,0.0,...,1364.430,-0.800080,-0.073112,9.6000,8.777160e-11,0,0,-1,1382,6
9681,0.000000,-0.000033,0.000027,-0.001410,0.000099,-0.000046,0.165468,-11.930100,0.0,0.0,...,1364.430,-0.463528,0.346568,4.4800,8.777160e-11,0,0,-1,1383,6
9682,0.000000,-0.000033,0.000027,-0.001410,0.000099,-0.000046,0.165468,-11.930100,0.0,0.0,...,0.000,-0.463528,0.346568,4.4800,8.777990e-11,0,0,-1,1384,6


In [40]:
'''I think the best way to do things might be to make two new columns, one with the normal vector in it, the other with all 9 grad(U) components, then
make a method that does the required linear algebra'''

df['grad_U'] = df[['grad(U):0', 'grad(U):1', 'grad(U):2', 'grad(U):3', 'grad(U):4', 'grad(U):5', 'grad(U):6', 'grad(U):7', 'grad(U):8']].values.tolist()
df['grad_U'] = df['grad_U'].apply(np.array)
#df['GRADU'] = df['GRADU'].as_matrix().reshape(2,2)
df['grad_U']

df['normals'] = df[['Normals:0', 'Normals:1', 'Normals:2']].values.tolist()
df['normals'] = df['normals'].apply(np.array)

# df['GRADU2'] = df[['grad(U):5', 'grad(U):6']].values.tolist()
# df['GRADU2'] = df['GRADU2'].apply(np.array)
# #df['grad(U):0']

# df['test'] = df['GRADU']+df['GRADU2']
#df['test']

In [101]:
def calculate_integrand(row):
    '''
    this method works out the vector which is used as the integrand for the surface integrals.
    
    It also multiplies it by the area of the cell so we can then sum the integrands up
    to approximate the surface integral
    
    '''
    gradU = row['grad_U'].reshape(3, 3)
    norm_v = row['normals'].reshape(3, 1)
    p = row['p']
    area = row['Area']
    #print(np.shape(norm_v))
    
#     x = np.dot(np.arange(9).reshape(3,3), norm_v)*5
#     print(x)
#     print(x[1])
    sigma = gradU + gradU.T - p*np.identity(3)
    
    integrand = np.dot(sigma, norm_v)*area
    #sys.exit()
    return integrand

In [102]:
df['surface_integrand']=df.apply(calculate_integrand, axis=1)

In [103]:
df

,p,dUdt:0,dUdt:1,dUdt:2,U:0,U:1,U:2,grad(U):0,grad(U):1,grad(U):2,...,grad(U):8,Area,Normals:0,Normals:1,Normals:2,Cell ID,Block Number,grad_U,normals,surface_integrand
0,0.917458,0.000000,0.000002,-0.000065,0.000000,-0.000002,0.004778,-0.408749,0.0,0.0,...,0.5016,4.375000e-11,1,0,0,0,2,"[-0.408749, 0.0, 0.0, -0.486604, -0.3457030000...","[1, 0, 0]","[[-7.5904325e-11], [-2.1288925e-11], [-2.15677..."
1,0.917458,0.000000,0.000002,-0.000065,0.000000,-0.000002,0.004778,-0.408749,0.0,0.0,...,0.5016,4.375000e-11,1,0,0,1,2,"[-0.408749, 0.0, 0.0, -0.331276, -0.4801109999...","[1, 0, 0]","[[-7.5904325e-11], [-1.4493325000000002e-11], ..."
2,0.917991,0.000000,0.000006,-0.000169,0.000000,-0.000009,0.013578,-0.965296,0.0,0.0,...,0.7920,4.375000e-11,1,0,0,2,2,"[-0.9652959999999999, 0.0, 0.0, -0.54505500000...","[1, 0, 0]","[[-1.2462550625e-10], [-2.3846156250000006e-11..."
3,0.918500,0.000000,0.000009,-0.000241,0.000000,-0.000013,0.021010,-1.324450,0.0,0.0,...,0.8720,4.375000e-11,1,0,0,3,2,"[-1.32445, 0.0, 0.0, -0.358157, -0.444338, 979...","[1, 0, 0]","[[-1.5607375e-10], [-1.566936875e-11], [-5.099..."
4,0.918913,0.000000,0.000011,-0.000292,0.000000,-0.000015,0.027297,-1.567000,0.0,0.0,...,0.8960,4.375000e-11,1,0,0,4,2,"[-1.567, 0.0, 0.0, -0.245414, -0.198614, 829.2...","[1, 0, 0]","[[-1.7731494375000003e-10], [-1.07368625000000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9679,0.000000,-0.000056,0.000076,-0.002668,0.000185,-0.000098,0.136725,-22.161400,0.0,0.0,...,9.6000,8.777990e-11,0,0,-1,1381,6,"[-22.1614, 0.0, 0.0, 0.0, 0.0, 0.0, -0.80008, ...","[0, 0, -1]","[[7.0230942392e-11], [6.4177640488e-12], [-1.6..."
9680,0.000000,-0.000056,0.000076,-0.002668,0.000185,-0.000098,0.136725,-22.161400,0.0,0.0,...,9.6000,8.777160e-11,0,0,-1,1382,6,"[-22.1614, 0.0, 0.0, -4.04714, 2.50932, 1364.4...","[0, 0, -1]","[[7.0224301728e-11], [-1.197517870307808e-07],..."
9681,0.000000,-0.000033,0.000027,-0.001410,0.000099,-0.000046,0.165468,-11.930100,0.0,0.0,...,4.4800,8.777160e-11,0,0,-1,1383,6,"[-11.9301, 0.0, 0.0, -4.04714, 2.50932, 1364.4...","[0, 0, -1]","[[4.0684594204800004e-11], [-1.197886230158688..."
9682,0.000000,-0.000033,0.000027,-0.001410,0.000099,-0.000046,0.165468,-11.930100,0.0,0.0,...,4.4800,8.777990e-11,0,0,-1,1384,6,"[-11.9301, 0.0, 0.0, 0.0, 0.0, 0.0, -0.463528,...","[0, 0, -1]","[[4.06884414872e-11], [-3.04217043832e-11], [-..."


In [104]:
#to calculate the integral, simply sum up the surface_integrand column
#this works because all of the integrand values have been scaled by the cell area.
z = df['surface_integrand'].sum()
print(z)

[[-3.93642927e-08]
 [ 1.71721970e-06]
 [-2.38349944e-04]]


In [109]:
def scale_w(row):
    return row['U:1']*row['Area']

In [110]:
df['scaled_w']=df.apply(scale_w, axis=1)

In [111]:
y = df['scaled_w'].sum()
print(y)

-1.3199860090381197e-11
